In [5]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

### SPARK 

In [6]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

### Load data 

In [7]:
data = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET, hive)

In [8]:
item_dict = load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_ITEM_DICT, hive)

### Create target dataset 

In [10]:
product_item = item_dict.filter("product <> '' and product <> 'none'")\
                        .select("item_id", "product", "item_user_count")\
                        .distinct()

In [13]:
target_items = product_item.filter(f.col("product").isin(PRODUCT_LIST)).toPandas()["item_id"].to_list()

In [15]:
len(target_items)

13

In [16]:
item_dict.filter(f.col("item_id").isin(target_items)) \
         .groupBy("item_id").agg(f.first("product"),
                                 f.first("item_user_count")).show(13)

,item_id,"first(product, false)","first(item_user_count, false)"
0,214250791,mybuch_online_ip,296
1,214300159,rabota_ru,2849
2,214230946,kredit,5203
3,214238693,leasing,449
4,214314599,sberrating,2606
5,214337742,two_gis,308
6,214228698,korpkarta,1802
7,214240895,lyurist,2362
8,214204148,credit_card,2303
9,214224178,evotor,823


In [17]:
target_dataset = data.filter(f.col("item_id").isin(target_items))

In [19]:
target_dataset.filter("user_id = 34").show(3)

,user_id,item_id,timestamp,rating
0,34,214224178,None,0.0
1,34,214243813,None,0.0
2,34,214226594,None,0.0


In [20]:
target_dataset.count()

12608765

### SAVE DATA 

In [22]:
drop_table(SBX_TEAM_DIGITCAMP,RECSYS_STORY_DATASET_TARGET , hive)
create_table_from_df(SBX_TEAM_DIGITCAMP, RECSYS_STORY_DATASET_TARGET, target_dataset, hive)

In [23]:
load_table(SBX_TEAM_DIGITCAMP,RECSYS_STORY_DATASET_TARGET , hive).count()

12608765

In [ ]:
RECSYS_STORY_DATASET_TARGET